In [7]:
'''
Inputs:
start end value: the start vertex, the end vertex and the weight of the edge.
e.g. 0 1 10, from 0 to 1 costs 10 units.
'''
num_vtx = 6
edges = [{i: 0} for i in range(num_vtx + 1)]
edges

[{0: 0}, {1: 0}, {2: 0}, {3: 0}, {4: 0}, {5: 0}, {6: 0}]

In [ ]:
def my_add_edge(direct_graph = False):
    '''
    Inputs:
    1. start: the start vertex.
    2. end: the end vertex. 
    3. value: the weight of the edge.
    e.g. 0 1 10, from 0 to 1 costs 10 units.

    Note:
    my_add_edge ends while input is -1.
    '''
    while True:
        start = int(input("start vtx: "))
        if start == -1:
            break       
        end = int(input("end vtx: "))
        assert start > 0
        assert end < num_vtx + 1
        cost = int(input("cost: "))
        edges[start][end] = cost
        if not direct_graph:
            edges[end][start] = cost
my_add_edge(direct_graph=False)

In [ ]:
def p1_add_edge(direct_graph = False):
    
    '''
    Inputs:
    edgeIndex edgeWeight nameOfVertexU nameOfVertexV
    e.g. (e1 1 v1 v2)
    1. idx: edgeIndex.
    2. w: edgeWeight. 
    3. u: nameOfVertexU.
    4. v: nameOfVertexV.
    e.g. e1 1 v1 v2, description: edge 1, starting from v1 to v2, weights 1 unit.

    Note:
    p1_add_edge ends while input is nothing, i.e. empty.
    '''

    while True:
        edge = input("Enter an edge, e.g.(e1 1 v1 v2): ")
        if edge == "":
            break
        idx, w, u, v = edge.split()
        # parse inputs to useful information
        w = int(w)
        u = int(u[1:]) # remove char 'v'
        v = int(v[1:]) # remove char 'v'

        edges[u][v] = w

        if not direct_graph:
            edges[v][u] = w


p1_add_edge(direct_graph=False)

In [ ]:
''' 
In hw 1, edges = [{0: 0},
 {1: 0, 2: 1, 5: 4, 6: 2},
 {2: 0, 5: 5, 4: 8, 1: 1},
 {3: 0, 4: 1, 5: 5, 6: 6},
 {4: 0, 6: 2, 2: 8, 3: 1},
 {5: 0, 6: 3, 1: 4, 2: 5, 3: 5},
 {6: 0, 1: 2, 3: 6, 4: 2, 5: 3}]
'''
edges = [{0: 0},
 {1: 0, 2: 1, 5: 4, 6: 2},
 {2: 0, 5: 5, 4: 8, 1: 1},
 {3: 0, 4: 1, 5: 5, 6: 6},
 {4: 0, 6: 2, 2: 8, 3: 1},
 {5: 0, 6: 3, 1: 4, 2: 5, 3: 5},
 {6: 0, 1: 2, 3: 6, 4: 2, 5: 3}]

In [10]:
# implement 1, using naive approach
# using permuations to find all possible paths

import math
import numpy as np
from itertools import permutations

def create_graph(edges):
    graph = np.asarray([[np.inf] * (num_vtx + 1)] * (num_vtx + 1))

    for u, u_edges in enumerate(edges):

        for v, w in u_edges.items():
            graph[u][v] = w
    
    return graph


# implementation of traveling Salesman Problem
def travellingSalesmanProblem(graph, src):
    # store all vertex apart from src vertex
    vertex = [i for i in range(1, num_vtx + 1)]

    # store minimum weight Hamiltonian Cycle
    min_cost = np.inf
    min_path = []
    perms = permutations(vertex)
    for perm in perms:

        # store current Path weight(cost)
        cur_path_weight = 0

        # compute current path weight
        
        s = perm[0] 
        k = s
        for v in perm:
            cur_path_weight += graph[k][v]
            k = v
        cur_path_weight += graph[k][s]

        # update minimum
        if min_cost > cur_path_weight:
            min_cost = cur_path_weight
            min_path = perm
    
    return min_path, min_cost

graph = create_graph(edges=edges)
min_path, min_cost = travellingSalesmanProblem(graph, 1)
print(f"from implement 1: min_path={min_path}, min_cost={min_cost}")

from implement 1: min_path=(1, 2, 5, 3, 4, 6), min_cost=16.0


In [12]:
# implement 2, using dp approach
# record and update weights from each cities to other cities

import math
import numpy as np
from itertools import permutations

def create_graph(edges):
    graph = np.asarray([[np.inf] * (num_vtx + 1)] * (num_vtx + 1))

    for u, u_edges in enumerate(edges):

        for v, w in u_edges.items():
            graph[u][v] = w
    
    return graph


# implementation of traveling Salesman Problem
def travellingSalesmanProblem(graph, src):

    print(f"original graph: \n {graph}")
    # store all vertex apart from src vertex
    vertex = [i for i in range(1, num_vtx + 1)]

    # store minimum weight Hamiltonian Cycle
    min_path = np.inf
    
    # From i to j, if there exists a path via k whose cost is lower than directly from i to j,
    # we will choose path i -> k -> j, and update graph[i][j] = graph[i][k] + graph[k][j].
    for n_iter in range(num_vtx + 1):
        # print(f'#{n_iter}')
        # print(graph)
        for i in range(1, num_vtx + 1):
            for j in range(i, num_vtx + 1):
                for k in range(1, num_vtx + 1):
                    via_k = graph[i][k] + graph[k][j]

                    if graph[i][j] > via_k:
                        graph[i][j] = min(graph[i][j], via_k)
                        graph[j][i] = graph[i][j]
                        

	
    print(f"after hamiltonian cycle solver: \n {graph}")
    
    path = [src]
    min_total_cost = 0
    cur = src
    for it in range(num_vtx-1):
        m_idx, m_cost = -1, np.inf
        for i, c in enumerate(graph[cur]):
            if i not in path and c < m_cost:
                m_cost = c
                m_idx = i
        path.append(m_idx)
        min_total_cost += m_cost
        print(f"#iter{it}, graph[{cur}]={graph[cur]}, next city idx={m_idx}, cost={m_cost}, total_cost={min_total_cost}")
        cur = m_idx
    
    path.append(src)
    min_total_cost += graph[cur][src]
    print(f"#iter{num_vtx-1}, graph[{cur}]={graph[cur]}, next city idx={src}, cost={graph[cur][src]}, total_cost={min_total_cost}\n\n")



    
    return path, min_total_cost

graph = create_graph(edges=edges)
min_path, min_cost = travellingSalesmanProblem(graph, 1)
print(f"from implement 2: min_path={min_path}, min_cost={min_cost}")

original graph: 
 [[ 0. inf inf inf inf inf inf]
 [inf  0.  1. inf inf  4.  2.]
 [inf  1.  0. inf  8.  5. inf]
 [inf inf inf  0.  1.  5.  6.]
 [inf inf  8.  1.  0. inf  2.]
 [inf  4.  5.  5. inf  0.  3.]
 [inf  2. inf  6.  2.  3.  0.]]
after hamiltonian cycle solver: 
 [[ 0. inf inf inf inf inf inf]
 [inf  0.  1.  5.  4.  4.  2.]
 [inf  1.  0.  6.  5.  5.  3.]
 [inf  5.  6.  0.  1.  5.  3.]
 [inf  4.  5.  1.  0.  5.  2.]
 [inf  4.  5.  5.  5.  0.  3.]
 [inf  2.  3.  3.  2.  3.  0.]]
#iter0, graph[1]=[inf  0.  1.  5.  4.  4.  2.], next city idx=2, cost=1.0, total_cost=1.0
#iter1, graph[2]=[inf  1.  0.  6.  5.  5.  3.], next city idx=6, cost=3.0, total_cost=4.0
#iter2, graph[6]=[inf  2.  3.  3.  2.  3.  0.], next city idx=4, cost=2.0, total_cost=6.0
#iter3, graph[4]=[inf  4.  5.  1.  0.  5.  2.], next city idx=3, cost=1.0, total_cost=7.0
#iter4, graph[3]=[inf  5.  6.  0.  1.  5.  3.], next city idx=5, cost=5.0, total_cost=12.0
#iter5, graph[5]=[inf  4.  5.  5.  5.  0.  3.], next city idx

In [7]:
# implement 3

n = num_vtx # there are four nodes in example graph (graph is 1-based)

MAX_INT = 10**9
# dist[i][j] represents shortest distance to go from i to j
# this matrix can be calculated for any given graph using
# all-pair shortest path algorithms
graph = create_graph(edges=edges)
dist = graph

# memoization for top down recursion
memo = np.asarray([[-1] * (1 << (n+1))] * (n+1))
memo.astype(np.int64)


def fun(i, mask):
	# base case
	# if only ith bit and 1st bit is set in our mask,
	# it implies we have visited all other nodes already
	if mask == ((1 << i) | 3):
		return dist[1][i]

	# memoization
	if memo[i][mask] != -1:
		return memo[i][mask]

	res =  MAX_INT # result of this sub-problem

	# we have to travel all nodes j in mask and end the path at ith node
	# so for every node j in mask, recursively calculate cost of
	# travelling all nodes in mask
	# except i and then travel back from node j to node i taking
	# the shortest path take the minimum of all possible j nodes
	for j in range(1, n+1):
		if (mask & (1 << j)) != 0 and j != i and j != 1:
			res = min(res, fun(j, mask & (~(1 << i))) + dist[j][i])
	memo[i][mask] = res # storing the minimum value
	return res


# Driver program to test above logic
ans = MAX_INT
for i in range(1, n+1):
	# try to go from node 1 visiting all nodes in between to i
	# then return from i taking the shortest route to 1
	ans = min(ans, fun(i, (1 << (n+1))-1) + dist[i][1])

print("The cost of most efficient tour = " + str(ans))

The cost of most efficient tour = 16.0
